In [2]:
# Instalação do inspyred
!pip install inspyred

     |████████████████████████████████| 88 kB 3.9 MB/s 


In [ ]:
from random import Random
from time import time
from inspyred import ec
from inspyred.ec import terminators
import numpy as np

# Gerar populações


def generate_(random, args):
    size = args.get("num_inputs", 12)
    # Máximo 16.000 central
    return [random.randint(0, 16000) for i in range(size)]

# função para avaliar soluções


def evaluate_(candidates, args):
    fitness = []
    for cs in candidates:
        fit = perform_fitness(cs[0], cs[1], cs[2], cs[3], cs[4],
                              cs[5], cs[6], cs[7], cs[8], cs[9], cs[10], cs[11])
        fitness.append(fit)
    return fitness

# Faz a função de fitness para cada candidato
def perform_fitness(carga1_D, carga1_C, carga1_T, carga2_D, carga2_C, carga2_T, carga3_D, carga3_C, carga3_T, carga4_D, carga4_C, carga4_T):
    # Arredondando
    carga1_D = np.round(carga1_D)  # Dianteiro
    carga1_C = np.round(carga1_C)  # Central
    carga1_T = np.round(carga1_T)  # Traseiro

    carga2_D = np.round(carga2_D)  # Dianteiro
    carga2_C = np.round(carga2_C)  # Central
    carga2_T = np.round(carga2_T)  # Traseiro

    carga3_D = np.round(carga3_D)  # Dianteiro
    carga3_C = np.round(carga3_C)  # Central
    carga3_T = np.round(carga3_T)  # Traseiro

    carga4_D = np.round(carga4_D)  # Dianteiro
    carga4_C = np.round(carga4_C)  # Central
    carga4_T = np.round(carga4_T)  # Traseiro

    # C  | t  | m3/t | Lucro 
    # C1 | 18 | 480  | 310   
    # C2 | 15 | 650  | 380    
    # C3 | 23 | 580  | 350    
    # C4 | 12 | 390  | 285   

    # Peso x Lucro
    # 18*310 + 15*380 + 23*350 + 12 * 285 = 22750
    # 22750: Estimativa de valor superior
    lucro_max = 22750
    num_h = 13

    fit = float(((0.310 * carga1_D + 0.310 * carga1_C + 0.310 * carga1_T) +
                (0.380 * carga2_D + 0.380 * carga2_C + 0.380 * carga2_T) +
                (0.350 * carga3_D + 0.350 * carga3_C + 0.350 * carga3_T) +
                (0.285 * carga4_D + 0.285 * carga4_C + 0.285 * carga4_T)) / lucro_max)

    # Compartimento | Capacidade (t) | Capacidade vol
    #   Dianteiro   |     10         |    6800
    #   Central     |     16         |    8700
    #   Traseiro    |      8         |    5300

    # Restrição de carga por setor
    h1 = np.maximum(0, float((carga1_D + carga2_D + carga3_D +
                    carga4_D) - 10000)) / float(10000 / num_h)  # Comp dianteiro: peso máx 10.000

    h2 = h2 = np.maximum(0, float(
        (carga1_C + carga2_C + carga3_C + carga4_C) - 16000)) / float(16000 / num_h)  # Comp central: peso máx 16.000

    h3 = np.maximum(0, float((carga1_T + carga2_T + carga3_T + carga4_T) - 8000)
                    ) / float(8000 / num_h)  # Comp traseiro: peso máx 8.000

    # Restrição volume da carga 
    h4 = np.maximum(0, float((0.480 * carga1_D + 0.650 * carga2_D +
                    0.580 * carga3_D + 0.390 * carga4_D) - 6800)) / float(6800 / num_h)  # Comp dianteiro: vol máx 6800m3

    h5 = np.maximum(0, float((0.480 * carga1_C + 0.650 * carga2_C +
                    0.580 * carga3_C + 0.390 * carga4_C) - 8700)) / float(8700 / num_h)  # Comp central: vol máx 8700m3

    h6 = np.maximum(0, float((0.48 * carga1_T + 0.65 * carga2_T +
                    0.58 * carga3_T + 0.39 * carga4_T) - 5300)) / float(5300 / num_h)  # Comp traseiro: vol máx 5300m3


    peso_max = 34000
    # Proporção de cargas
    prop_dianteira = float(10000 / peso_max)  # Máximo 10t
    prop_central = float(16000 / peso_max)  # Máximo 16t
    prop_traseira = float(8000 / peso_max)  # Máximo 8t

    carga_dianteira = float(carga1_D + carga2_D + carga3_D + carga4_D)
    carga_central = float(carga1_C + carga2_C + carga3_C + carga4_C)
    carga_traseira = float(carga1_T + carga2_T + carga3_T + carga4_T)
    soma_total_cargas = float(carga_dianteira + carga_central + carga_traseira)

    # Restrição de carga por proporção
    h7 = np.maximum(0, float(((carga_dianteira / soma_total_cargas) - prop_dianteira))
                    ) / float(prop_dianteira / num_h)  # Comp dianteiro
    h8 = np.maximum(0, float(((carga_central / soma_total_cargas) -
                              prop_central))) / float(prop_central / num_h)  # Comp central

    h9 = np.maximum(0, float(((carga_traseira / soma_total_cargas) -
                    prop_traseira))) / float(prop_traseira / num_h)  # Comp traseiro

    # Restrição por peso máximo da carga
    h10 = np.maximum(0, float((carga1_D + carga1_C + carga1_T) - 18000)
                     ) / float(18000 / 13)  # Peso máximo: 18.000
    h11 = np.maximum(0, float((carga2_D + carga2_C + carga2_T) - 15000)
                     ) / float(15000 / 13)  # Peso máximo: 15.000
    h12 = np.maximum(0, float((carga3_D + carga3_C + carga3_T) - 23000)
                     ) / float(23000 / 13)  # Peso máximo: 23.000
    h13 = np.maximum(0, float((carga4_D + carga4_C + carga4_T) - 12000)
                     ) / float(12000 / 13)  # Peso máximo: 12.000

    fit = fit - (h1 + h2 + h3 + h4 + h5 + h6 + h7 +
                 h8 + h9 + h10 + h11 + h12 + h13)

    return fit


def solution_evaluation(carga1_D, carga1_C, carga1_T, carga2_D, carga2_C, carga2_T, carga3_D, carga3_C, carga3_T, carga4_D, carga4_C, carga4_T):
    # Arredondando
    carga1_D = np.round(carga1_D)  # Dianteiro
    carga1_C = np.round(carga1_C)  # Central
    carga1_T = np.round(carga1_T)  # Traseiro

    carga2_D = np.round(carga2_D)  # Dianteiro
    carga2_C = np.round(carga2_C)  # Central
    carga2_T = np.round(carga2_T)  # Traseiro

    carga3_D = np.round(carga3_D)  # Dianteiro
    carga3_C = np.round(carga3_C)  # Central
    carga3_T = np.round(carga3_T)  # Traseiro

    carga4_D = np.round(carga4_D)  # Dianteiro
    carga4_C = np.round(carga4_C)  # Central
    carga4_T = np.round(carga4_T)  # Traseiro

    compartimento_dianteiro = carga1_D + carga2_D + carga3_D + carga4_D
    compartimento_central = carga1_C + carga2_C + carga3_C + carga4_C
    compartimento_traseiro = carga1_T + carga2_T + carga3_T + carga4_T
    total = compartimento_dianteiro + compartimento_central + compartimento_traseiro

    print("----- RESUMO -----")
    print("\n --- Peso por compartimento --- \n")
    print("Carga 1 Dianteiro:", carga1_D)
    print("Carga 1 Central:", carga1_C)
    print("Carga 1 Traseiro:", carga1_T)
    total_carga1 = float(carga1_D + carga1_C + carga1_T)
    print("Total carga 1", (total_carga1))

    print("\nCarga 2 Dianteiro:", carga2_D)
    print("Carga 2 Central:", carga2_C)
    print("Carga 2 Traseiro:", carga2_T)
    total_carga2 = float(carga2_D + carga2_C + carga2_T)
    print("Tota carga 2:", (total_carga2))

    print("\nCarga 3 Dianteiro:", carga3_D)
    print("Carga 3 Central:", carga3_C)
    print("Carga 3 Traseiro:", carga3_T)
    total_carga3 = float(carga3_D + carga3_C + carga3_T)
    print("Total carga 3:", (total_carga3))

    print("\nCarga 4 Dianteiro:", carga4_D)
    print("Carga 4 Central:", carga4_C)
    print("Carga 4 Traseiro:", carga4_T)
    total_carga4 = float(carga4_D + carga4_C + carga4_T)
    print("Total carga 4:", (total_carga4))

    cargas_total = float(total_carga1 + total_carga2 +
                         total_carga3 + total_carga4)

    print("\nPeso total do carregamento: ", cargas_total)
    print("\nCarga dianteira total:", carga1_D + carga2_D + carga3_D + carga4_D)
    print("Carga central total:", carga1_C + carga2_C + carga3_C + carga4_C)
    print("Carga traseira total:", carga1_T + carga2_T + carga3_T + carga4_T)

    # Peso da carga * volume
    volume_dianteiro = float((carga1_D * 0.480)+(carga2_D * 0.650) +
                             (carga3_D * 0.580) + (carga4_D * 0.390))
    volume_central = float(
        (carga1_C * 0.480) + (carga2_C * 0.650) + (carga3_C * 0.580) + (carga4_C * 0.390))
    volume_traseiro = float(
        (carga1_T * 0.480)+(carga2_T * 0.650) + (carga3_T * 0.580) + (carga4_T * 0.390))

    print("\n --- Volume por compartimento --- ")
    print("Total de volume dianteiro : ", volume_dianteiro)
    print("Total de volume central : ", volume_central)
    print("Total de volume traseiro : ", volume_traseiro)

    print("\n --- Proporção --- ")
    print("Proporção de Carga Dianteira: ", round(
        ((carga1_D + carga2_D + carga3_D + carga4_D) / cargas_total),3))
    print("Proporção de Carga Central: ", round(
        ((carga1_C + carga2_C + carga3_C + carga4_C) / cargas_total),3))
    print("Proporção de Carga Traseira: ", round(
        ((carga1_T + carga2_T + carga3_T + carga4_T) / cargas_total),3))

    lucro_c1 = float(0.310 * carga1_D + 0.310 * carga1_C + 0.310 * carga1_T)
    lucro_c2 = float(0.380 * carga2_D + 0.380 * carga2_C + 0.380 * carga2_T)
    lucro_c3 = float(0.350 * carga3_D + 0.350 * carga3_C + 0.350 * carga3_T)
    lucro_c4 = float(0.285 * carga4_D + 0.285 * carga4_C + 0.285 * carga4_T)

    print("\n --- Lucro por carga --- \n")
    print("Lucro carga 1:", lucro_c1)
    print("Lucro carga 2:", lucro_c2)
    print("Lucro carga 3:", lucro_c3)
    print("Lucro carga 4:", lucro_c4)
    print("Lucro Total :", lucro_c1 + lucro_c2 + lucro_c3 + lucro_c4)

    if (((compartimento_dianteiro/total) >= 0.3) or ((compartimento_dianteiro/total) <= 0.29)):
        print(
            f"proporção dianteira excedida: {np.round((compartimento_dianteiro/total),4)}")
    if (((compartimento_central/total) >= 0.48) or ((compartimento_central/total) <= 0.47)):
        print(
            f"proporção central excedida: {np.round((compartimento_central/total),4)}")
    if (((compartimento_traseiro/total) >= 0.24) or ((compartimento_traseiro/total) <= 0.23)):
        print(
            f"proporção traseiro excedida: {np.round((compartimento_traseiro/total),4)}")


def main():
    rand = Random()
    rand.seed(int(time()))  # inicar semente aleatoria

    ea = ec.GA(rand)
    ea.selector = ec.selectors.tournament_selection  # metodo de seleção: torneio
    ea.variator = [ec.variators.uniform_crossover,  # op genético crossover uniforme
                   ec.variators.gaussian_mutation]  # op genético mutação gaussian_mutation

    # determina os sobreviventes da prox geração
    ea.replacer = ec.replacers.steady_state_replacement

    # critério de parada: atingir o máx de geraçoes
    ea.terminator = terminators.generation_termination

    # geração de estatísticas da evolução
    ea.observer = [ec.observers.stats_observer, ec.observers.file_observer]

    final_pop = ea.evolve(generator=generate_,  # Função que gera a população
                          evaluator=evaluate_,  # Função que avalia as soluções
                          pop_size=10000,  # Tamanho da população
                          maximize=True,  # False:minimização
                          bounder=ec.Bounder(0, 16000),
                          max_generations=5000,  # Qtd de gerações
                          num_inputs=12,  # Qtd de genes (cromossomos)
                          crossover_rate=1.0,  # Taxa de cruzamento
                          num_crossover_points=1,   # Número de cortes do cruzamento
                          mutation_rate=0.25,  # Taxa de mutação
                          num_elites=1,  # numero de individuos elites a serem
                          # selecionados para a póxima população
                          num_selected=12,  # Numero de individuos
                          tournament_size=12,  # Torneio aletorio
                          #statistcs_fize=open("plane.csv", "w"),
                          #individuals_file=open("plane-individuals.csv", "w")
                          )

    

    # Melhor população por primeiro 0 = melhor individuo
    final_pop.sort(reverse=True)  # população final
    print("\n --- Melhor Individuo --- \n")
    print(final_pop[0],"\n")

    # Fitness final
    perform_fitness(final_pop[0].candidate[0], final_pop[0].candidate[1], final_pop[0].candidate[2], final_pop[0].candidate[3], final_pop[0].candidate[4], final_pop[0].candidate[5],
                    final_pop[0].candidate[6], final_pop[0].candidate[7], final_pop[0].candidate[8], final_pop[0].candidate[9], final_pop[0].candidate[10], final_pop[0].candidate[11])
    solution_evaluation(final_pop[0].candidate[0], final_pop[0].candidate[1], final_pop[0].candidate[2], final_pop[0].candidate[3], final_pop[0].candidate[4], final_pop[0].candidate[5],
                        final_pop[0].candidate[6], final_pop[0].candidate[7], final_pop[0].candidate[8], final_pop[0].candidate[9], final_pop[0].candidate[10], final_pop[0].candidate[11])


main()
